In [1]:
import spacy
import numpy
import sense2vec
import random

from numpy import dot
from numpy.linalg import norm
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
nlp = spacy.load('en')
model = sense2vec.load()

import os
os.environ['STANFORD_PARSER'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser.jar'
os.environ['STANFORD_MODELS'] = 'C:\\stanford-parser-full-2016-10-31\\stanford-parser-3.7.0-models.jar'
from nltk.parse.stanford import StanfordParser
cons_parser = StanfordParser(model_path = 'edu\\stanford\\nlp\\models\\lexparser\\englishPCFG.ser.gz')

# Load and Prepare Dataset

In [6]:
#read_path = 'data/conll2003/eng.train'
read_path = 'train.txt'
with open(read_path, "r") as ins:
    raw_data = []
    label_data = []
    
    temp_sent = ''
    temp_label = ''
    tkn = False
    
    for line in ins:
        #array.append(line)

        if len(line) == 1:
            raw_data.append(temp_sent)
            label_data.append(temp_label)
            temp_sent = ''
            temp_label = ''
            tkn = False
        else:
            if tkn == True:
                temp_sent += ' '
                temp_label += ' '
                
            temp_sent += line.split()[0] # 단어
            temp_label += line.split()[-1] # NER 라벨
            tkn = True

In [ ]:
sent_raw = []
sent_label = []
cnt = 0
for i, row in enumerate(raw_data):
    row_nlp = nlp(row)

#     if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
#         temp = []
#         for token in row_nlp:
#             temp.append(token.pos_) # 각 token의 pos 저장
#         if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
#             if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
#                 sent_raw.append(raw_data[i])
#                 sent_label.append(label_data[i]) 
 
    
    
    if raw_data[i][-1] == '.' or raw_data[i][-1] == '"': # 마지막에 쉼표가 있는 문장들만 선별하자.
        if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
            temp = []
            for token in row_nlp:
                temp.append(token.pos_) # 각 token의 pos 저장
            if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
                if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
                    sent_raw.append(raw_data[i])
                    sent_label.append(label_data[i]) 
#                 else:
#                     cnt += 1


In [ ]:
len(sent_raw)

In [22]:
from data_handling_for_heuristic import *
#read_path = 'data/conll2003/eng.train'
read_path = 'data_logic_warehouse/type_2/merged_replace.txt'
#read_path = 'train.txt'
sent_raw, sent_label = load_conll2003(read_path)

In [23]:
### Preprocessing

def del_ent_in_list(delete_list, X):
# token 단위로 확인하면서 삭제한다
# call to ( boycott ) 이면 (과 )는 삭제된다. token단위이므로... (token단위 기준: space)
# X: ['EU rejects German call to boycott British lamb .', ... ]
# Y: ['I-ORG O I-MISC O O O I-MISC O O', ... ]
# delete_list = [')','(', ';', ...]
    i = 0
    while i < len(X):
        if any(str(X[i]) in t for t in delete_list):
            #del Y[i]
            del X[i]
        #else:
        i+=1               
    return X #, Y

def del_ent_in_string(delete_list, X):
# char 단위로 확인하면서 삭제한다
# 예를 들어, boycott이 아니라 boy(cott이면 해당 token은 삭제된다. 이처럼 char레벨까지 체크한다.
# X: ['EU rejects German call to boycott British lamb .', ... ]
# Y: ['I-ORG O I-MISC O O O I-MISC O O', ... ]
# delete_list = [')','(', ';', ...]    
    i=0
    while i < len(X):
        #print(str(sp_test[i]))
        list_X = list(X[i])
        if len(list_X)==1:
            break
        for char in list_X:
            if any(char in t for t in delete_list):
                #print(X)
                #print(Y)
                #del Y[i]
                del X[i]
                break
        #else:
        i+=1
    return X #, Y

In [24]:
global index_SBAR
global list_SBAR 

# index_SBAR = -1
# list_SBAR = [0] * len(sent_raw[3].split())

def traverseTree(tree, temp_sentence):

    global index_SBAR
    global list_SBAR    
    
#     if tree.label() == 'ROOT': # 처음에 initialization
#         index_SBAR = -1
#         list_SBAR = [0] * len(sent_list)

    #print("tree:", tree)
    #print("======> tree.pos()", tree.pos())
    #print("======> tree.height()", tree.height())
    #print("======> tree.label()", tree.label())
    #print("======> tree.leaves()", tree.leaves())
    
    #print("======> current_depth", current_depth)
    #print('======> current depth list', list_current_depth)

    #print('*************************************************************************************************')    
    #print('\n')

    if type(tree) == nltk.tree.Tree:
        
        # Initialication
        current_height = tree.height()

    
        ### 로직 한계점
        ### 만약 SBAR안에 SBAR이 있는 경우 구분하지 못한다. 또는 SBAR이 또다른 SBAR과 바로 옆에 붙어있는 경우 구분하지 못한다.
        ### 그러면 나중에 제일 첫 글자만 참고해서 어떤 부사절인지 보는데, 만약 2개 이상 SBAR이 포함되어있거나 붙어있는 경우
        ### index가 제일 앞에 있는 SBAR만 참고하게 된다. (예를 들어, because SBAR이 뒤에 있는 경우 무시하게 된다)
        
        # SBAR
        if tree.label() == 'SBAR':
            #print('SBAR start!!!')
            
            if len(tree.leaves()) <= 1: # SBAR token개수가 1개 이하이면 return. 
                return None
            
            for i, token in enumerate(temp_sentence):
                #print('temp_sentence[i]:', temp_sentence[i], ', tree.leaves()[0]:',tree.leaves()[0]) 
                #print('temp_sentence[i+1]:', temp_sentence[i+1], ', tree.leaves()[1]:', tree.leaves()[1])
                
                
                #if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+len(tree.leaves())-1] == tree.leaves()[-1]:
                if temp_sentence[i] == tree.leaves()[0] and temp_sentence[i+1] == tree.leaves()[1]:
                    #print('break!!!!!!!!!!!!!!!!!!!!!!')
                    index_SBAR = i
                    break
                
            if not index_SBAR == -1:
                for j in range(index_SBAR, index_SBAR+len(tree.leaves())):
                    list_SBAR[j] = 1   
                    
            return None

    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            traverseTree(subtree, temp_sentence) # recursive

In [25]:
def re_numbering(list_SBAR):
# ex. [1,1,1,0,0,1,1,0,1,1] => [1,1,1,0,0,2,2,0,3,3]
# 후처리하기 쉽게하기 위해 numbering을 다시한다.
    
    cur_idx = 1
    new_list_SBAR = [0] * len(list_SBAR)
    
    for i, _ in enumerate(list_SBAR):
        if list_SBAR[i]!=0:
            new_list_SBAR[i] = cur_idx
        if not i==len(list_SBAR)-1:
            if list_SBAR[i]!=0 and list_SBAR[i+1]==0:
                cur_idx += 1
    return new_list_SBAR

In [26]:
def what_clause(renumbered_list_SBAR, token_raw_list):

    clause_list = ['none'] * len(renumbered_list_SBAR)
    sent_clauses = ['because', 'before', 'until', 'after', 'while', 'if', 'since', 'when', 'as',
                   'Because', 'Before', 'Until', 'After', 'While', 'If', 'Since', 'When', 'As']
    
    for i, num in enumerate(renumbered_list_SBAR):
        if i==0 and renumbered_list_SBAR[i]!=0:
            if token_raw_list[i] in sent_clauses:
                clause_list[i] = token_raw_list[i]
        if renumbered_list_SBAR[i-1]==0 and renumbered_list_SBAR[i]!=0:
            if token_raw_list[i] in sent_clauses:
                #print(token_raw_list[i])
                clause_list[i] = token_raw_list[i]    

    temp_index = -1
    for i, num in enumerate(renumbered_list_SBAR):
        if clause_list[i]!='none':
            temp_clause = clause_list[i]
            temp_index = renumbered_list_SBAR[i]
        if renumbered_list_SBAR[i] == temp_index:
            clause_list[i] = temp_clause       
                
    return clause_list

In [27]:
# test = [1, 1, 1, 0, 0 ,0]
# sent = ['I', 'am', 'a', 'student', '.', 'hi']
# label = ['O', 'O', 'O', 'I-LOC','O','O']

# temp = []
# for i, x in enumerate(test):
#     if x == 1:
#         temp.append(sent[i])
#         end_pos = i+1

# print(end_pos)
# print(sent[:end_pos])
# print(sent[end_pos:])

# deleted_sent = sent[:end_pos]
# remained_sent = sent[end_pos:]

# deleted_label = label[:end_pos]
# remained_label = label[end_pos:]

# print(remained_sent + deleted_sent)
# print(remained_label + deleted_label)
# # del_sent = set(sent) - set(temp)
# # del_sent = list(del_sent)
# # del_sent += temp
# # del_sent

In [28]:
# test = [0, 0, 0, 1, 1 ,1]
# sent = ['I', 'am', 'a', 'student', '.', 'hi']
# label = ['O', 'O', 'O', 'I-LOC','O','O']

# temp = []
# tkn = False
# for i, x in enumerate(test):
#     if x == 1:
#         temp.append(sent[i])
        
#         if tkn == False:
#             start_pos = i
#             tkn = True

        
# print(start_pos)
# print(sent[:start_pos])
# print(sent[start_pos:])

# remained_sent = sent[:start_pos]
# deleted_sent = sent[start_pos:]

# remained_label = label[:start_pos]
# deleted_label = label[start_pos:]


# print(deleted_sent + remained_sent)
# print(deleted_label + remained_label)

In [29]:
global cnt_headmove
global cnt_tailmove

cnt_headmove = 0 
cnt_tailmove = 0 

### 양끝에 있는 SBAR(sent_clause)만 대상으로 하자. 가운데 있는 SBAR(sent_clause)은 제외하자. 
### 가운데 SBAR(sent_clause)은 보통 복잡한 문장인 경우가 많은데 이들은 의미를 많이 왜곡할 수 있으므로 제외하자.
### 꼬리부분의 SBAR(sent_clause)은 머리부분으로,  머리부분의 SBAR(sent_clause)은 꼬리부분으로 이동시키자.

### 양끝 모두 SBAR(sent_clause)가 있는 경우 제외하자.
def move_with_SBAR(list_SBAR, token_raw_list, token_label_list, clause_list):
    
    global cnt_headmove
    global cnt_tailmove
        
    # 주의: list_SBAR, sent_raw_list, sent_label_list의 길이는 모두 같다.
    sent_clauses = ['because', 'before', 'until', 'after', 'while', 'if', 'since', 'when', 'as',
                   'Because', 'Before', 'Until', 'After', 'While', 'If', 'Since', 'When', 'As']
    new_raw_sent_list = [] # 문장 여러개
    new_label_sent_list = []
    
    #print(token_raw_list)
    #print(list_SBAR)
    #print('------------------------------------------')    

    
    for i, _ in enumerate(list_SBAR):

        
        ### 문장 머리에 있을 경우
        #######################
        if i==0 and list_SBAR[i] != 0 and clause_list[i] in sent_clauses: # 문장 머리에 sent clause SBAR이 있는 경우..
            
            sbar_num = list_SBAR[i]
            temp = []
            for i, x in enumerate(list_SBAR):
                if x == sbar_num:
                    temp.append(token_raw_list[i])
                    end_pos = i+1

            deleted_sent = token_raw_list[:end_pos]
            remained_sent = token_raw_list[end_pos:]

            deleted_label = token_label_list[:end_pos]
            remained_label = token_label_list[end_pos:]

            #print(remained_sent + deleted_sent)
            #print(remained_label + deleted_label)

            cnt_headmove += 1
            temp_sent = remained_sent + deleted_sent
            temp_label = remained_label + deleted_label
            new_raw_sent_list.append((temp_sent))
            new_label_sent_list.append((temp_label))
        
        
        ### 문장 꼬리에 있을 경우
        #######################
        if i==len(list_SBAR)-2 and list_SBAR[i] !=0 and clause_list[i] in sent_clauses: # 문장 꼬리에 sent clause SBAR이 있는 경우..
            
            sbar_num = list_SBAR[i]
            temp = []
            tkn = False
            for i, x in enumerate(list_SBAR):
                if x == sbar_num:
                    temp.append(token_raw_list[i])

                    if tkn == False:
                        start_pos = i
                        tkn = True

            remained_sent = token_raw_list[:start_pos]
            deleted_sent = token_raw_list[start_pos:]

            remained_label = token_label_list[:start_pos]
            deleted_label = token_label_list[start_pos:]

            #print(deleted_sent + remained_sent)
            #print(deleted_label + remained_label)               

            cnt_tailmove += 1
            temp_sent = deleted_sent + remained_sent
            temp_label = deleted_label + remained_label
            new_raw_sent_list.append((temp_sent))
            new_label_sent_list.append((temp_label))

    return new_raw_sent_list, new_label_sent_list

# Main

In [30]:
# 괄호가 있어도 상관은 없는데..
# tree.leaves()에서는 괄호는 출력되지 않는다.
# 하지만1, string안에 괄호가 있는 경우는 parser 오류가 난다. 예를 들어 ap(ple인 단어가 있으면 오류가 난다. 
# 하지만2, 괄호가 있지만, 쌍으로 존재하지 않을경우 parser 오류가 난다. 예를 들어, ), ( 이렇게 한 개씩만 있는 경우...
# 즉, 한 문장에서 괄호가 pair로 존재하지 않는 경우는 삭제하자.!!!

In [31]:
new_sent_list = []
new_label_list = [] 

for i, _ in enumerate(sent_raw):
    
#     print('\n')
#     print(sent_raw[i])
    
    token_raw_list = sent_raw[i].split()
    token_label_list = sent_label[i].split()
    
    parenthesis_list = []
    for i, _ in enumerate(token_raw_list):
        if token_raw_list[i]=='(' or token_raw_list[i]==')':
            parenthesis_list.append((token_raw_list[i], i)) # append (pair)    
    
    ### preprocessing 
    pre_raw_list = del_ent_in_list(['(',')',';'], token_raw_list)
    pre2_raw_list = del_ent_in_string(['(',')'], token_raw_list)
    ### pos tagging
    pos_sent_raw_list = nltk.pos_tag(pre2_raw_list)
    #pos_sent_raw_list = nltk.pos_tag(token_raw_list)
    ### parsing
    parse_result = cons_parser.tagged_parse(pos_sent_raw_list)


    ###################################
    """ SBAR 이동 """
    ###################################
    # initialization 
    index_SBAR = -1
    list_SBAR = [0] * len(token_raw_list)    
    
    for tree in parse_result:
        parse_tree = tree
        traverseTree(parse_tree, token_raw_list)

    ###################################
    """ 괄호 복구 작업 """
    ###################################
    for pair in parenthesis_list: # 순서대로 리스트업 했기 때문에 순서대로 insert하기만 하면 됨. 
        parenthesis = pair[0]
        index = pair[1]
        # token_raw_list에 괄호 복구
        token_raw_list.insert(index, parenthesis)
        # SBAR list에 괄호 복구
        if parenthesis == '(':
            if list_SBAR[index+1] == 1:
                list_SBAR.insert(index, 1)
            else:
                list_SBAR.insert(index, 0)
        elif parenthesis == ')':
            if list_SBAR[index-1] == 1:
                list_SBAR.insert(index, 1)
            else:
                list_SBAR.insert(index, 0)        

    ##############################################
    """ Boundary 검사: 엔티티가 쪼개져 있으면 수정 """    
    ##############################################
    # parsing 결과를 보정
    tkn = True
    while(tkn):
        
        btn = False
        for i, num in enumerate(list_SBAR):
            if i!=len(list_SBAR)-1:
                if list_SBAR[i] == 1 and list_SBAR[i+1] == 0:
                    btn = True
        if btn == False and list_SBAR[-1] == 1:
            tkn = False

        if not 1 in list_SBAR:
            tkn = False
            
        for i, num in enumerate(list_SBAR):
            if i!=0: # 첫 번째 index는 제외
                if list_SBAR[i-1] == 1 and list_SBAR[i] == 0: # find SBAR boundary!
                    if token_label_list[i-1] != 'O': # 엔티티라면
                        if token_label_list[i].split('-')[0] == 'I': # (좀 더 구체적으로..) INSIDE 계열의 엔티티라면..
                            list_SBAR[i] = 1
                        else: # BEGIN 계열이면 그냥 pass
                            tkn = False # break
                    else: # 엔티티가 아니라면
                        tkn = False # break             
    
#     ### for printf 
#     print(list_SBAR)
#     if 1 in list_SBAR:
#         temp = ''
#         for k, tkn in enumerate(token_raw_list):
#             if list_SBAR[k] == 1:
#                 temp = temp+token_raw_list[k]+' '
#         print(temp)
                
    # generation with SBAR
    if 1 in list_SBAR: # 현재 문장에서 SBAR이 있으면...
        # renumbering
        list_SBAR = re_numbering(list_SBAR)
        clause_list = what_clause(list_SBAR, token_raw_list)
#         print(len(clause_list), len(list_SBAR))
#         print(clause_list)
#         print(list_SBAR)

#         temp = ''
#         for k, tkn in enumerate(token_raw_list):
#             if list_SBAR[k] == 1:
#                 temp = temp+token_raw_list[k]+' '
#         print(temp)        
        
        
        new_sent_raw_list, new_sent_label_list = move_with_SBAR(list_SBAR, token_raw_list, token_label_list, clause_list)

        if len(new_sent_raw_list) != 0: # 새로운 문장이 생성되었는가?
            # 주의: append가 아니라 +로해야됨. new_sent_list가 list이기 때문에
            new_sent_list += new_sent_raw_list
            new_label_list += new_sent_label_list
            
#             ### for printf 
#             print('***********************************')
#             print(sent_raw[i])            
            
#             ### for printf 
#             print(list_SBAR)
#             if 1 in list_SBAR:
#                 temp = ''
#                 for k, tkn in enumerate(token_raw_list):
#                     if list_SBAR[k] == 1:
#                         temp = temp+token_raw_list[k]+' '
#                 print(temp)            
            
            
            

    
    # 엔티티 포함되어 있으면 냅둔다.


#     for i, _ in enumerate(list_SBAR):

#         if not i==0:
#             if list_SBAR[i-1]==0 and list_SBAR[i]==1:

#         else: # i==0일 때...

#         print('------------------------------------------------')            
#         print(sent_raw[i])
#         print('------------------------------------------------')
#         print(temp)
#         print(list_SBAR)
#         print('\n')
#         print('------------------------------------------------')
    

In [32]:
print(len(new_sent_list), len(new_label_list))
print(cnt_headmove)
print(cnt_tailmove)

722 722
68
654


# Store new sent

In [33]:
len(new_sent_list)

722

In [34]:
path_write = 'data_logic_warehouse/type_2/merged_reduce_move_'+str(len(new_sent_list))+'.txt'

with open(path_write, 'w', encoding='UTF-8') as txt:
    for i, new_token_list in enumerate(new_sent_list):

        #splited_sent = new_sent_list[i].split()
        #splited_label = new_label_list[i].split()
        for j, token in enumerate(new_token_list):
            txt.write(str(new_sent_list[i][j])+' '+'NNP'+' '+'B-NP'+' '+str(new_label_list[i][j]))
            txt.write('\n')

        txt.write('\n')

In [4]:
a = ['a','b','c','d']
b = ['e','f','a']

print(any((True for x in a if x in b)))

True


In [11]:
def filtering_clauses(raw_data, label_data):
    sent_clauses = ['because', 'before', 'until', 'after', 'while', 'if', 'since', 'when', 'as', 'Because', 'Before', 'Until', 'After', 'While', 'If', 'Since', 'When', 'As']
    new_data = []
    new_label = []
    
    for i, _ in enumerate(raw_data):
        if type(raw_data[i])!=type([]):
            raw_data[i] = raw_data[i].split()
        if any((True for x in raw_data[i] if x in sent_clauses))==True:
            new_data.append(raw_data[i])
            new_label.append(label_data[i])
    return new_data, new_label

In [13]:
a, b = filtering_clauses(raw_data, label_data)

In [15]:
a

[['The',
  'European',
  'Commission',
  'said',
  'on',
  'Thursday',
  'it',
  'disagreed',
  'with',
  'German',
  'advice',
  'to',
  'consumers',
  'to',
  'shun',
  'British',
  'lamb',
  'until',
  'scientists',
  'determine',
  'whether',
  'mad',
  'cow',
  'disease',
  'can',
  'be',
  'transmitted',
  'to',
  'sheep',
  '.'],
 ['Germany',
  "'s",
  'representative',
  'to',
  'the',
  'European',
  'Union',
  "'s",
  'veterinary',
  'committee',
  'Werner',
  'Zwingmann',
  'said',
  'on',
  'Wednesday',
  'consumers',
  'should',
  'buy',
  'sheepmeat',
  'from',
  'countries',
  'other',
  'than',
  'Britain',
  'until',
  'the',
  'scientific',
  'advice',
  'was',
  'clearer',
  '.'],
 ['"',
  'We',
  'do',
  "n't",
  'support',
  'any',
  'such',
  'recommendation',
  'because',
  'we',
  'do',
  "n't",
  'see',
  'any',
  'grounds',
  'for',
  'it',
  ',',
  '"',
  'the',
  'Commission',
  "'s",
  'chief',
  'spokesman',
  'Nikolaus',
  'van',
  'der',
  'Pas',
  'told